### Import Packages

In [1]:
import pandas as pd
import numpy as np
from ENID.acquisition import *
from ENID.interpolation import *
from alerce.core import Alerce
from time import gmtime, strftime
import matplotlib.pyplot as plt
import torch.nn as nn
import george
import random

random.seed(1)

### Import Source Names

In [2]:
datasources = pd.read_csv('sources.csv',header=None) # Give your sources in a .csv file with the ZTF-names. An example with blue continuum sources is attached

### Initialisation

In [3]:
# Model wanted
ModelWeights = 'Models/Simple GRU/Weights'

# Name of the saved files
Version = strftime("%Y%m%d%H%M%S",gmtime()) #Default used UTC time and date.

# Number of points before
timediff = 14
noofpoint = 2

# Plotting?
plotbool = 'n'

### Download Lightcurves

In [4]:
##### Retrieve light curves part

# Get ready by making it a list anc checking that they are not floats
ztf_raw = list(datasources[0])
ztf_names = [x for x in ztf_raw if type(x) != float]

# Find the sources on ALeRCE
print('Number of sources :', len(ztf_raw), '\n')
alerce_found, alerce_missing = source_search_alerce(ztf_names)

# Tell how many were found
print('Found ', alerce_found.shape[0], 'objects')
print('Missing ', len(alerce_missing), 'objects')

if len(alerce_missing) > 0:
    for mis in alerce_missing:
        print('Missing ', alerce_missing['oid'][mis])

# Get the object id for each source
sources = list(alerce_found['oid'])

# Retrieve the lightcurves for the sources
object_dictionary = {"Name": [], "Data": [], "Label": []}

for i in range(len(sources)):
    
    
    lightcurve = lc_compile(sources[i])
    if len(lightcurve['R_mag']) > 1 and len(lightcurve['G_mag']) > 1:
        object_dictionary['Data'].append(lightcurve)
        object_dictionary['Name'].append(ztf_names[i])
        object_dictionary['Label'].append(['SN II', 1])
    else:
        print('Warning : Not enought detections. Ignoring entry.')
        

    
    
save_file = open("pickle_lightcurves.pickle", mode='wb')
pickle.dump(object_dictionary, save_file)
save_file.close()    

Number of sources : 55 

Found  55 objects
Missing  0 objects

Importing lightcurve and metadata for  ZTF19abzwbxy

Importing lightcurve and metadata for  ZTF18abtuzsb

Importing lightcurve and metadata for  ZTF18abklbam

Importing lightcurve and metadata for  ZTF18abmdpwe

Importing lightcurve and metadata for  ZTF18abqbuaj

Importing lightcurve and metadata for  ZTF18abqcsdq

Importing lightcurve and metadata for  ZTF18abthaii

Importing lightcurve and metadata for  ZTF18ablwafp

Importing lightcurve and metadata for  ZTF18abvzdvj

Importing lightcurve and metadata for  ZTF18abwkrbl

Importing lightcurve and metadata for  ZTF18acurqaw

Importing lightcurve and metadata for  ZTF18acbxskh

Importing lightcurve and metadata for  ZTF18acrwheu

Importing lightcurve and metadata for  ZTF18acszayr

Importing lightcurve and metadata for  ZTF18adaykvg

Importing lightcurve and metadata for  ZTF18adazblo

Importing lightcurve and metadata for  ZTF19aamowaf

Importing lightcurve and metadata fo

### Interpolate Lightcurves

In [5]:
file = open('pickle_lightcurves.pickle', "rb")
datadict = pickle.load(file)
dummy_dict, source_idx = initialise(datadict,timediff,noofpoint)

processed_data = preprocessing('pickle_lightcurves.pickle',Version,timediff,noofpoint)

Saving Arrays...


In [7]:
datafile = 'Data/'+Version+'/data_lc.npy'
labelsfile = 'Data/'+Version+'/labels.npy'

data = np.load(datafile)
labels = np.load(labelsfile)

print('Data Dimensions :', data.shape)
print('Label Dimensions :', labels.shape)

x = torch.permute(torch.tensor(data), [0,2,1])

Data Dimensions : (54, 2, 200)
Label Dimensions : (54, 2)


### Build Model & Import Weights

In [8]:
class simple_ENID(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(simple_ENID, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.GRU = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, bidirectional=False) 
        self.Dense = nn.Linear(hidden_dim, output_dim)
        self.predict = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        x_, h_ = self.GRU(x)
        x_ = self.Dense(x_[:,-1])
        x_ = self.predict(x_)
        
        return x_
    
# Model Building
input_dim = x.shape[2]
num_classes = 6
hidden_dim = 64
n_layers = 1

Net = simple_ENID(input_dim=input_dim, hidden_dim =hidden_dim, output_dim=num_classes, n_layers=n_layers)
print(Net)

Net.load_state_dict(torch.load(ModelWeights))

simple_ENID(
  (GRU): GRU(2, 64, batch_first=True)
  (Dense): Linear(in_features=64, out_features=6, bias=True)
  (predict): Softmax(dim=1)
)


<All keys matched successfully>

### Make Predictions

In [9]:
classes = ['CV', 'SN II', 'SN Ia', 'SLSN-I', 'SN IIn', 'SN Ib/c']

pred = Net(x.float())

label = np.argmax(pred.detach().numpy(), axis = 1)
conf = np.max(pred.detach().numpy(), axis = 1)

for i in range(len(label)):
    print('Class:', classes[label[i]], ' -  Confidence:', conf[i])

Class: CV  -  Confidence: 0.9999999
Class: CV  -  Confidence: 0.9999999
Class: SN Ib/c  -  Confidence: 0.92862153
Class: SN II  -  Confidence: 0.8502242
Class: SN Ia  -  Confidence: 0.66323805
Class: CV  -  Confidence: 0.9999902
Class: SN Ib/c  -  Confidence: 0.99895775
Class: SN IIn  -  Confidence: 0.979807
Class: SN Ib/c  -  Confidence: 0.8655839
Class: SN Ib/c  -  Confidence: 0.8580053
Class: SN IIn  -  Confidence: 0.9735744
Class: SN II  -  Confidence: 0.97399694
Class: SN II  -  Confidence: 0.9437273
Class: SN Ia  -  Confidence: 0.9890342
Class: SN Ia  -  Confidence: 0.9656921
Class: SN IIn  -  Confidence: 0.99991703
Class: SN II  -  Confidence: 0.999818
Class: SN Ib/c  -  Confidence: 0.8886919
Class: SN Ib/c  -  Confidence: 0.9753405
Class: SN II  -  Confidence: 0.99945694
Class: SN II  -  Confidence: 1.0
Class: SN IIn  -  Confidence: 0.97051716
Class: SN IIn  -  Confidence: 0.99580634
Class: SN Ib/c  -  Confidence: 0.76370376
Class: SN II  -  Confidence: 0.999915
Class: SN II  -